In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab  
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
	
height = 419
width = 1
channels = 1
n_inputs = height * width

conv1_fmaps = 16
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 32
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

conv3_fmaps = 64
conv3_ksize = 3
conv3_stride = 2
conv3_pad = "SAME"


pool3_fmaps = conv2_fmaps


n_fc1 = 150
n_outputs = 2

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")
    keep_prob= tf.placeholder(tf.float32, shape=[], name="regularizer")

    X_drop=tf.nn.dropout(X,keep_prob)
#conv1_ksize size of filter
#conv1_stride step of filter
#padding weather boundary is exist 

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=[conv1_ksize,1],
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.elu, name="conv1")
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 1, 1], strides=[1, 2, 1, 1], padding="VALID")


conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=[conv2_ksize,1],
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.elu, name="conv2")
conv2_drop=tf.nn.dropout(conv2,keep_prob)
pool2 = tf.nn.max_pool(conv2_drop, ksize=[1, 2, 1, 1], strides=[1, 2, 1, 1], padding="VALID")

conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=[conv3_ksize,1],
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.elu, name="conv3")
conv3_drop=tf.nn.dropout(conv3,keep_prob)
pool3 = tf.nn.max_pool(conv3_drop, ksize=[1, 2, 1, 1], strides=[1, 2, 1, 1], padding="VALID")

pool3_flat = tf.reshape(pool3, shape=[-1, pool3.shape[1]*pool3.shape[2]*pool3.shape[3]])




with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.elu, name="fc1")
    fc1_drop=tf.nn.dropout(fc1,keep_prob)
with tf.name_scope("output"):
    logits = tf.layers.dense(fc1_drop, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y,name='obj-function')
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter("logs/", sess.graph)    
print(X)
print(X_reshaped)
print(conv1)
print(conv2)
print(pool3)
print(pool3_flat)


def Scaler(x1):
    x=x1[1:]
    maximun=max(x)
    minimun=min(x)
    for i in range(len(x)):
        x[i]=(x[i]-minimun)/(maximun-minimun)
        x1[i+1]=x[i]*0.99+0.01
    return x1
    
def prodata(train_data_list):
    data=[]
    for record in train_data_list:
        all_value=record.split(',')
        #print((all_value))
        inputs=(np.asfarray(all_value))
        #inputs=Scaler(inputs)
        data.append(inputs)
    return np.array(data)



data_file=open('traindata_r.dat','r')
data_list=data_file.readlines()
data_file.close()

test_data=prodata(data_list)


X_test = test_data[:,1:].astype(np.float32)
y_test = test_data[:,0].astype(np.int32)




# saver = tf.train.Saver()
# with tf.Session() as sess:
#     saver.restore(sess,'model/RFTN3-2.21.ckpt')

#     n_epochs = 40
#     accuracy_test = sess.run(accuracy,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
#     proba = sess.run(Y_proba,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
#     logit_out = sess.run(logits,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
# print(accuracy_test)


Tensor("inputs/X:0", shape=(?, 419), dtype=float32)
Tensor("inputs/Reshape:0", shape=(?, 419, 1, 1), dtype=float32)
Tensor("conv1/Elu:0", shape=(?, 419, 1, 32), dtype=float32)
Tensor("conv2/Elu:0", shape=(?, 210, 1, 64), dtype=float32)
Tensor("pool3/MaxPool:0", shape=(?, 105, 1, 64), dtype=float32)
Tensor("pool3/Reshape:0", shape=(?, 6720), dtype=float32)


In [15]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,'model/RFTN3-2.21.ckpt')
    n_epochs = 40
    accuracy_test = sess.run(accuracy,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
    proba = sess.run(Y_proba,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
    logit_out = sess.run(logits,feed_dict={X: X_test, y: y_test,keep_prob:1.0})
    print(accuracy_test)

INFO:tensorflow:Restoring parameters from model/RFTN3-2.21.ckpt
0.9633795
